In [ ]:
import numpy as np
import pandas as pd
import pyfixest as pf
from IPython.display import display, FileLink
from tabout import MTable,  DTable, ETable, BTable
from IPython.display import FileLink
import pylatex as pl 

# Use pylatex to create a tex file with the table
def make_pdf(tab_or_tabs, file):
    """
    Create a PDF document with one or more LaTeX tables.
    Accepts a single table string or a list of table strings.
    """
    doc = pl.Document()
    doc.packages.append(pl.Package("booktabs"))
    doc.packages.append(pl.Package("threeparttable"))
    doc.packages.append(pl.Package("makecell"))
    doc.packages.append(pl.Package("tabularx")) 
    doc.packages.append(pl.Package("array"))

    if not isinstance(tab_or_tabs, list):
        tab_or_tabs = [tab_or_tabs]

    with doc.create(pl.Section("PyFixest LaTeX Tables")):
        for i, tab in enumerate(tab_or_tabs, 1):
            with doc.create(pl.Table(position="htbp")) as table:
                table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)



# Create a multiindex dataframe with random data
row_index = pd.MultiIndex.from_tuples(
    [
        ("Gr 1", "Var 1"),
        ("Gr 1", "Var 2"),
        ("Gr 1", "Var 3"),
        ("Gr 2", "Var 4"),
        ("Gr 2", "Var 5"),
        ("Gr 3", "Var 6"),
    ]
)

col_index = pd.MultiIndex.from_product([["France", "US"], ["High", "Low"]])
dfa = pd.DataFrame(np.random.randn(6, 4).round(3), index=row_index, columns=col_index)


tabs=[]
tabs.append(MTable(dfa, notes="Test notes...").make(type="tex", tab_width=None))  
tabs.append(
    MTable(dfa).make(type="tex", tab_width="linewidth",
                     tex_style={
                                "arraystretch": 1,
                                "tabcolsep": "1pt",
                                "x_col_align": "left",
                                "data_align": "l",
                                "header_addlinespace": "0ex",
                                "data_addlinespace": "0ex",
                                "group_header_format": r"\bfseries %s",
                                "notes_fontsize_cmd": r"\footnotesize",
                            },)
                    )
#tabs.append(MTable(dfa).make(type="tex", tab_width="10cm"))
#tabs.append(MTable(dfa).make(type="tex", tab_width=r"0.8\textwidth"))

# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tabs, "docs/SampleTableDoc2")
display(FileLink("docs/SampleTableDoc2.pdf"))


c:\PythonProjects\tabout\docs\SampleTableDoc2.pdf